#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [17]:
import pandas as pd
import pymongo

In [18]:
client = pymongo.MongoClient('mongodb://mongo:27017')
database = client['exercices']
collection = database['kickstarter']

In [19]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [20]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  int64  
 1   name              149998 non-null  object 
 2   category          150000 non-null  object 
 3   main_category     150000 non-null  object 
 4   currency          150000 non-null  object 
 5   deadline          150000 non-null  object 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  object 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  object 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  object 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), int64(1), object(12)
memory usage: 16.0+ MB


In [21]:
#df_ks=df_ks.astype({'goal': 'float64'})
#df_ks=df_ks.astype({'pledged': 'float64'})
#df_ks=df_ks.astype({'backers': 'float64'})
#df_ks=df_ks.astype({'usd pledged': 'float64'})
df_ks=df_ks.convert_dtypes()
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  Int64  
 1   name              149998 non-null  string 
 2   category          150000 non-null  string 
 3   main_category     150000 non-null  string 
 4   currency          150000 non-null  string 
 5   deadline          150000 non-null  string 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  string 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  string 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  string 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: Int64(1), float64(1), object(4), string(8)
memory usage: 16.2+ MB


### Importer les données

In [22]:
import json
data_json = json.loads(df_ks.to_json(orient='records'))
collection.insert_many(data_json)

## Question 1

In [27]:
None

## Question 2  

In [24]:
liste = []
for i in collection.find({"state": "successful"}) :
    liste.append(i)
len(liste)


106080

## Question 3

In [25]:
categories = collection.distinct("category")
print(categories)

['3D Printing', 'Academic', 'Accessories', 'Action', 'Animals', 'Animation', 'Anthologies', 'Apparel', 'Apps', 'Architecture', 'Art', 'Art Books', 'Audio', 'Bacon', 'Blues', 'Calendars', 'Camera Equipment', 'Candles', 'Ceramics', "Children's Books", 'Childrenswear', 'Chiptune', 'Civic Design', 'Classical Music', 'Comedy', 'Comic Books', 'Comics', 'Community Gardens', 'Conceptual Art', 'Cookbooks', 'Country & Folk', 'Couture', 'Crafts', 'Crochet', 'DIY', 'DIY Electronics', 'Dance', 'Design', 'Digital Art', 'Documentary', 'Drama', 'Drinks', 'Electronic Music', 'Embroidery', 'Events', 'Experimental', 'Fabrication Tools', 'Faith', 'Family', 'Fantasy', "Farmer's Markets", 'Farms', 'Fashion', 'Festivals', 'Fiction', 'Film & Video', 'Fine Art', 'Flight', 'Food', 'Food Trucks', 'Footwear', 'Gadgets', 'Games', 'Gaming Hardware', 'Glass', 'Grace is Leaving', 'Graphic Design', 'Graphic Novels', 'Hardware', 'Hip-Hop', 'Horror', 'Illustration', 'Immersive', 'Indie Rock', 'Installations', 'Interacti

In [26]:
for i in categories :
    print(i, " : ", collection.find( { "category": i } ).count())

<ipython-input-26-405a1cc3077a>:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(i, " : ", collection.find( { "category": i } ).count())


3D Printing  :  542
Academic  :  734
Accessories  :  2408
Action  :  564
Animals  :  194
Animation  :  2034
Anthologies  :  600
Apparel  :  5654
Apps  :  5070
Architecture  :  600
Art  :  6716
Art Books  :  2130
Audio  :  328
Bacon  :  156
Blues  :  226
Calendars  :  224
Camera Equipment  :  330
Candles  :  336
Ceramics  :  256
Children's Books  :  5372
Childrenswear  :  384
Chiptune  :  20
Civic Design  :  260
Classical Music  :  2128
Comedy  :  1846
Comic Books  :  2064
Comics  :  3862
Community Gardens  :  230
Conceptual Art  :  786
Cookbooks  :  434
Country & Folk  :  3580
Couture  :  216
Crafts  :  3668
Crochet  :  122
DIY  :  960
DIY Electronics  :  716
Dance  :  1802
Design  :  3282
Digital Art  :  1048
Documentary  :  12996
Drama  :  1742
Drinks  :  1990
Electronic Music  :  1716
Embroidery  :  98
Events  :  644
Experimental  :  714
Fabrication Tools  :  192
Faith  :  878
Family  :  260
Fantasy  :  264
Farmer's Markets  :  350
Farms  :  964
Fashion  :  6758
Festivals  :  626
Fi

## Question 4

## Question 5

## Question 6 

In [29]:
cur=collection.find({"name" : {'$regex' : "Sport" }})
len(list(cur))

646